单独测试prompt；
1，对于原始问题，通过更多的问话来引导模型补充完整的内容；
2，对于解决方案，先让模型给出多个思路，然后综合自动生成的与原始的方案，给出一个更加完备的方案。
3，为了避免解决方案过于简单，再对它生成提供测试案例或者更多的问话，以这样的方式来完善其中的技术细节。
4，有了详细的问题描述和解决方案，其他的部分就相对容易了。


In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=3000)

In [2]:
raw_question = """使用ChatGPT可以生成能够正常运行的、不太复杂的代码，所以也可用于生成RPA代码（模拟人工操作来实现UI任务的自动化执行）。
一般的方法是编写一个比较详细的生成提示词，包含生成步骤和各种约束，让它直接生成最终代码，但是这样的生成过程难以控制，比如使用了不存在的库函数，
或者缺少必要的数据等等，也难以保证生成的流程可以正确执行。
为了保证实现方法足够简单，在提供解决方案时，除了chatgpt，请不要使用任何别的机器学习或者深度学习方法。"""
raw_solution = """基本思路是将生成RPA流程的复杂过程拆分为几个步骤，包括生成需求描述、生成主要步骤、生成每一个步骤的执行代码、生成每一个步骤的测试代码。
全部文档和代码生成完成之后，执行生成的测试代码，再根据错误提示来更新主步骤的代码，直接所有的测试代码完成。
这样，通过多步骤和多次迭代的方式，将复杂的RPA流程生成过程分解为多个更简单、具体的生成步骤，
既能充分利用chatgpt已经掌握的常识，还能够让几个不同部分的内容相互引用，进一步保证生成代码的有效性，即通过需求说明书生成代码，通过测试用例生成测试代码，通过测试代码来检验RPA流程代码。
"""

In [3]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
发明人的问题描述往往比较模糊，只集中于核心的概念和逻辑，对于问题相关的背景、价值和创新描述模糊不清。
请根据以下提示，为发明人的简单描述补充相关信息，重新生成一个更好的问题描述：
1，这个发明解决了什么具体的软件或计算问题吗？
2，那些用户、软件系统或计算环境会遇到这个问题？
3，你能举一个这个软件问题通常出现的例子吗？
4，这个问题如何影响软件系统或应用的操作、效率或用户体验？
5，目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者对用户不友好？
6，在技术领域或者行业发展中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？
7，你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？
8，这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？
9，这个软件问题是被广泛认知的，还是说你的发明揭示了它？
10，解决这个问题有没有任何特定的软件、硬件或系统约束？

发明人提供的问题描述: {raw_question}

系统的、详细的、完整的问题描述是：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [4]:
question = rewriteq_chain.run(raw_question)
print(question)

本发明旨在解决使用ChatGPT生成可运行的RPA（Robotic Process Automation）代码的问题。RPA是一种模拟人工操作来实现UI任务的自动化执行的技术，广泛应用于各种软件系统和计算环境中。 例如，一个典型的RPA任务可能是自动从网页上抓取数据并填充到Excel表格中。

然而，使用ChatGPT生成RPA代码的过程中存在一些关键的挑战。首先，生成代码的过程难以控制，例如，生成的代码可能使用了不存在的库函数，或者缺少必要的数据。其次，由于ChatGPT生成的代码可能存在错误，也难以保证生成的流程可以正确执行。

这些问题会严重影响软件系统或应用的操作、效率和用户体验。目前存在的解决方案，如提供更详细的生成提示词，虽然可以在一定程度上解决这个问题，但是仍然存在不足。例如，这种方法仍然无法保证生成的代码是正确的，也无法解决生成过程中的控制问题。

最近，随着机器学习和深度学习的发展，生成代码的技术变得越来越重要。在这种背景下，解决使用ChatGPT生成RPA代码的问题变得特别相关和紧迫。

为了量化这个问题，我们可以通过统计生成的代码中错误的数量，以及执行生成的流程的成功率来衡量其影响。这个问题不仅影响到开发人员，因为他们需要生成和维护RPA代码，也影响到终端用户，因为他们需要使用生成的代码来完成任务。

这个软件问题是广泛认知的，但是本发明揭示了使用ChatGPT生成RPA代码中的一些新的问题和挑战。在提供解决方案时，我们需要遵守一些特定的软件、硬件或系统约束，例如，我们不能使用任何别的机器学习或者深度学习方法。


In [5]:
template_commentq = '''你是一个著名的产品经理，请根据发明人提供的问题描述和自动扩充的问题描述，提取其中涉及到的核心概念、术语。
请提供这些概念、术语的精简描述，并说明它们之间可能存在的逻辑或者语义关系。对于复杂的技术或者过程，请提供示例来说明，以方便理解。

发明人提供的问题描述: {raw_question}
自动扩充的问题描述: {question}

概念和术语的介绍：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

commentq = commentq_chain.run({'raw_question': raw_question, 'question': question})
print(commentq)

1. ChatGPT: 这是一个使用深度学习技术训练的语言生成模型，用于生成人类般的文本。它可以理解和生成文本，用于各种应用，包括生成代码。

2. RPA（Robotic Process Automation）: 这是一种技术，用于通过模拟人的行为来自动执行UI任务。例如，它可以模拟人在网页上抓取数据并将其填充到Excel表格中。

3. 库函数：这是预先编写好的函数，存储在库中，可以在任何需要的地方调用。如果生成的代码使用了不存在的库函数，那么代码将无法运行。

4. 生成提示词：这是一种方法，用于指导ChatGPT生成代码。提示词可能包含生成步骤和各种约束，以帮助模型生成正确的代码。

5. 机器学习和深度学习：这些是人工智能的子领域，用于从数据中学习并做出预测或决策。在本问题中，发明人明确指出不应使用这些方法。

6. 错误的数量和成功率：这是两种评价生成代码质量的指标。错误的数量表示生成的代码中存在多少错误，而成功率表示生成的流程可以正确执行的频率。

关系：ChatGPT通过理解生成提示词来生成RPA代码，但生成过程可能会出现错误，如使用不存在的库函数或缺少必要的数据。这些错误会影响生成代码的质量，从而影响软件的操作、效率和用户体验。评价生成代码质量的方式是统计错误的数量和执行生成流程的成功率。在寻求解决方案时，需遵守一些约束，如不能使用任何其他机器学习或深度学习方法。


In [6]:
def run_prompt(llm, inputs, outputs, template, params):
    prompt = PromptTemplate(input_variables=inputs, template=template)
    chain = LLMChain(llm=llm, prompt=prompt, output_key=outputs)
    return chain.run(params)

In [8]:
template_refineq = '''你是一个专业的程序员，精通常用算法和数据结构，并有非常丰富的机器学习和深度学习开发经验。
你的任务是根据两个版本的问题描述，完善发明人提供的解决方案，补充解决方案中缺失的处理步骤，为每一步处理提供具体的算法实现。
请根据以下步骤来执行：
1，总结解决方案中涉及到的核心概念和术语；
2，根据对应的业务场景，设计对应的示例业务数据；
3，将发明人的解决方案转换为一系列处理步骤，每个处理步骤都有输入和输出数据；

最终的实现方案应该是具体的、详细的，任何程序员都可以按照这个实现方案将最终的功能实现出来。
同时，这个实现方案需要满足以下条件：
1，它包含核心数据和算法描述，与发明人的解决方案有相同的含义；
2，通过一系列处理过程，最初的业务数据转换成了想要结果数据；
3，每一个处理步骤只能使用最初的业务数据和前面步骤的输出数据；

简略的问题描述: {raw_question}
详细的问题描述: {question}
发明人的解决方案：{raw_solution}

具体的实现描述：'''

refineq = run_prompt(llm, ['raw_question', 'question', 'raw_solution'], 'refineq', template_refineq, 
                      {'raw_question': raw_question, 'question': question, 'raw_solution': raw_solution})
print(refineq)

1，核心概念和术语：
- ChatGPT：OpenAI的一个预训练模型，可用于生成人类一样的文本。
- RPA（Robotic Process Automation）：模拟人工操作来实现UI任务的自动化执行的技术。
- 测试代码：用于检验程序是否按预期运行的代码。

2，设计示例业务数据：
- 需求描述：自动从一个特定网页上提取所有的电子邮件地址，并保存到Excel表格中。
- 主要步骤：打开网页，找到所有的电子邮件地址，打开Excel，输入电子邮件地址。
- 执行代码：具体实现上述步骤的代码。
- 测试代码：测试执行代码是否正确的代码。

3，处理步骤：
- 步骤一：生成需求描述
  - 输入：业务需求
  - 输出：需求描述
  - 算法：利用ChatGPT生成需求描述
- 步骤二：生成主要步骤
  - 输入：需求描述
  - 输出：主要步骤
  - 算法：利用ChatGPT生成主要步骤
- 步骤三：生成每一个步骤的执行代码
  - 输入：主要步骤
  - 输出：执行代码
  - 算法：利用ChatGPT生成执行代码
- 步骤四：生成每一个步骤的测试代码
  - 输入：执行代码
  - 输出：测试代码
  - 算法：利用ChatGPT生成测试代码
- 步骤五：执行生成的测试代码
  - 输入：测试代码
  - 输出：测试结果
  - 算法：执行测试代码，根据错误提示更新主步骤的代码，直到所有的测试代码完成。

以上这个实现方案满足了前面提到的所有条件，包含了核心数据和算法描述，与发明人的解决方案有相同的含义，通过一系列处理过程，最初的业务数据转换成了想要结果数据，每一个处理步骤只能使用最初的业务数据和前面步骤的输出数据。


In [9]:
template_candidates = '''你们是三个著名的程序员，请基于问题描述、专利发明人提供的解决方案和实现过程描述，各自提供一个具体的且有创新性的解决方案。
生成的解决方案要满足以下要求：
1，解决方案可以转换为代码设计说明书，并且方便使用代码来实现；
2，每个解决方案要包含基本的解决思路，使用的算法和数据描述，技术分析，以及实现步骤。
3，技术分析应该包括所用的算法，以及论证这个算法能够解决当前问题的分析。
4，解决步骤不应该存在多余的步骤，同时也不能缺失必须的处理过程。
5，发明人提供的解决方案可以用于参考，也可以直接引用，只要最终的方案能够解决问题就可以。

待解决的问题: {question}
发明人的解决方案：{raw_solution}
实现过程描述：{refineq}

请一步步思考，确保解决方案能够解决上述问题，三个解决方案分别是：'''

candidates = run_prompt(llm, ['question', 'raw_solution', 'refineq'], 'candidates', template_candidates, 
                      {'question': question, 'raw_solution': raw_solution, 'refineq': refineq})
print(candidates)

解决方案一：
基本思路是生成一个上下文敏感的RPA代码生成器，通过ChatGPT生成伪代码，然后转化为具体的RPA代码。
- 使用的算法：基于transformer的自然语言处理模型，根据输入的需求描述生成初步的RPA伪代码。
- 数据描述：输入数据为用户提供的需求描述，输出数据为RPA代码。
- 技术分析：通过转化为伪代码的方式可以简化代码生成的复杂度，同时也可以通过生成的伪代码进行进一步的微调和优化。
- 实现步骤：
  - 用户输入需求描述
  - 使用ChatGPT生成对应的伪代码
  - 将伪代码转化为具体的RPA代码
  - 通过RPA系统运行生成的代码，进行验证和优化

解决方案二：
基本思路是使用ChatGPT生成一个RPA任务脚本的草稿，然后通过人机交互的方式进行优化和完善。
- 使用的算法：基于transformer的自然语言处理模型，根据输入的需求描述生成初步的RPA代码。
- 数据描述：输入数据为用户提供的需求描述，输出数据为RPA代码。
- 技术分析：通过人机交互的方式可以有效地处理生成代码的错误和缺失，同时也可以提高生成代码的成功率。
- 实现步骤：
  - 用户输入需求描述
  - 使用ChatGPT生成对应的RPA任务脚本草稿
  - 用户核对并优化生成的脚本
  - 用户确认并运行生成的脚本

解决方案三：
基本思路是通过ChatGPT生成RPA任务流程图，然后使用专门的工具将流程图转化为具体的RPA代码。
- 使用的算法：基于transformer的自然语言处理模型，根据输入的需求描述生成初步的RPA任务流程图。
- 数据描述：输入数据为用户提供的需求描述，输出数据为RPA任务流程图。
- 技术分析：通过生成流程图的方式可以更直观地展示RPA任务的执行过程，同时也可以通过专门的工具将流程图转化为具体的RPA代码。
- 实现步骤：
  - 用户输入需求描述
  - 使用ChatGPT生成对应的RPA任务流程图
  - 使用专门的工具将流程图转化为具体的RPA代码
  - 通过RPA系统运行生成的代码，进行验证和优化


In [11]:
template_rewritea = '''你是一个在知名大公司工作的软件系统架构师，你的任务是根据问题描述和多个解决方案，重新实现一个详细的、现实可行的实现方案。
这个解决方案需要满足以下条件：
1，它是可实现的，能够根据它来写出对应的代码；同时，它有创新性，可以用于专利申请；
2，它不需要考虑所有的可能性，只要能够解决最核心的问题即可。
3，请描述它的基本思路是什么？用到了哪些成熟算法和数据？
4，请具体分析这个解决思路是如何解决这个问题的。
5，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
6，创新点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
7，它的具体解决步骤是什么？可以综合多个解决方案中的实现，但要保证每个步骤都是必须的，而且能够解决核心问题。

问题描述: {question}
自动生成的解决方案: {candidates}
发明人提供的解决方案：{raw_solution}

请重新生成一个具备创新性的解决方案：'''


solution = run_prompt(llm, ['question', 'raw_solution', 'candidates'], 'solution', template_rewritea, 
                      {'question': question, 'raw_solution': raw_solution, 'candidates': candidates})
print(solution)

基于上述的几个解决方案，我们可以设计一个新的解决方案：基于ChatGPT的上下文敏感的RPA代码生成器，该生成器不仅能够生成RPA代码，还能生成对应的测试代码以及任务流程图。

基本思路：生成器会根据用户的需求描述，首先生成一个RPA任务流程图和对应的伪代码，然后将伪代码转化为具体的RPA代码，同时生成对应步骤的测试代码。这个过程会经历多次迭代，直到所有的测试代码都能成功执行。

使用的算法：基于transformer的自然语言处理模型，根据输入的需求描述生成RPA任务流程图和伪代码，再转化为具体的RPA代码和测试代码。

数据描述：输入数据为用户提供的需求描述，输出数据为RPA任务流程图，具体的RPA代码以及对应的测试代码。

具体解决步骤：
1. 用户输入需求描述
2. 使用ChatGPT生成对应的RPA任务流程图和伪代码
3. 将伪代码转化为具体的RPA代码以及对应的测试代码
4. 在RPA系统中运行生成的代码和测试代码，记录错误信息
5. 根据运行结果和错误信息，修正RPA代码和测试代码
6. 重复步骤4和步骤5，直到所有的测试代码都能成功执行。

这个方案与一般的解决方法不同在于，我们不仅生成RPA代码，同时也生成对应的测试代码，通过测试代码来检查和修正RPA代码，从而提高生成代码的质量和可靠性。

创新点对应的优势和价值：一方面，我们的方法能有效提高代码生成的质量和可靠性，减少生成代码的错误率，从而提高RPA任务的执行效率；另一方面，生成的任务流程图能帮助用户更好地理解和掌握RPA任务的执行过程，提高用户的工作效率。


In [12]:
template_reflextion = '''你是一个著名的程序员，请根据问题描述和选定的解决方案，回答以下问题：
1，可以根据这个解决方案来解决对应的问题吗？如果不能，请给出理由，并提供解决方法；
2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
4，针对解决方案的必须步骤，提供具体的实现说明；可以用伪代码来表示；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
5，在涉及算法和数据时，请提供具体的示例；

问题描述: {question}
解决方案: {solution}

针对解决方案的说明：'''


reflextion = run_prompt(llm, ['question', 'solution'], 'reflextion', template_reflextion, 
                      {'question': question, 'solution': solution})
print(reflextion)

1，可以根据这个解决方案来解决对应的问题吗？如果不能，请给出理由，并提供解决方法；
答：可以。这个解决方案的核心思想是生成RPA代码的同时，也生成对应的测试代码，通过测试来检查和修正RPA代码，从而提高代码的质量和可靠性。这种方法更偏向于测试驱动开发(TDD)，可以有效地提高代码的质量和减少错误。

2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
答：实现这个解决方案需要的数据主要是用户的需求描述。这些需求描述可以直接由用户提供，或者通过收集用户的操作行为和使用习惯来获取。获取这些数据的工作量主要取决于用户的数量和复杂性。

3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
答：在这个解决方案中，所有的步骤都是必要的。生成任务流程图、伪代码、RPA代码和测试代码是解决问题的核心步骤，运行和修正代码是保证代码质量的关键步骤。

4，针对解决方案的必须步骤，提供具体的实现说明；可以用伪代码来表示；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
答：以下是解决方案的具体实现说明：

```python
# 输入用户需求描述
user_requirement = input()

# 使用ChatGPT生成任务流程图和伪代码
flowchart, pseudocode = ChatGPT.generate(user_requirement)

# 将伪代码转化为RPA代码和测试代码
rpa_code, test_code = pseudocode_to_code(pseudocode)

# 在RPA系统中运行代码和测试代码
errors = run_and_test(rpa_code, test_code)

# 如果存在错误，则修正代码
while errors:
    rpa_code, test_code = correct_code(rpa_code, test_code, errors)
    errors = run_and_test(rpa_code, test_code)

# 返回生成的RPA代码
return rpa_code
```

5，在涉及算法和数据时，请提供具体的示例；
答：例如，用户的需求描述是"我想从网页上抓取一些数据并填充到Excel表格中"，ChatG

In [13]:
template_reflextion2 = '''你是一个著名的程序员，请根据问题描述和选定的解决方案，提供一个详细的代码实现说明书。
请根据以下提示，生成一份代码实现说明：
1，可以根据这个解决方案来实现相应的功能吗？如果不能，请给出理由，并补充解决方法；
2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
4，针对解决方案的必须步骤，提供具体的实现说明；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
5，在涉及算法时，请给出具体的算法名；在涉及到数据时，请提供一个示例；

问题描述: {question}
解决方案: {solution}

针对解决方案的说明：'''


reflextion = run_prompt(llm, ['question', 'solution'], 'reflextion', template_reflextion2, 
                      {'question': question, 'solution': solution})
print(reflextion)

1，解决方案的可行性：这个方案是可行的，因为ChatGPT是一个强大的自然语言生成模型，能够根据用户的需求描述生成相应的代码和测试代码。

2，需要的数据：输入数据为用户提供的需求描述，例如“我需要从一个网页上抓取数据并填充到Excel表格中”。输出数据为RPA任务流程图，具体的RPA代码以及对应的测试代码。由于输入数据来自用户，获取这些数据的工作量较小，只需设置一个用户输入界面即可。

3，非必要的步骤：这个解决方案中所有的步骤都是必要的。每一个步骤都在为最后的目标——生成可执行的、无错误的RPA代码和测试代码服务。

4，具体实现说明：
   - 步骤1：设置一个用户输入界面，让用户输入他们的需求描述。
   - 步骤2：使用ChatGPT生成对应的RPA任务流程图和伪代码。这可以通过调用OpenAI的GPT-3 API实现。输入为用户的需求描述，输出为RPA任务流程图和伪代码。
   - 步骤3：将伪代码转化为具体的RPA代码以及对应的测试代码。这可以通过编写一个转换函数实现，该函数接收伪代码为输入，输出RPA代码和测试代码。
   - 步骤4：在RPA系统中运行生成的代码和测试代码，记录错误信息。这需要一个RPA执行环境，可以使用现有的RPA工具，如UiPath、Automation Anywhere等。
   - 步骤5：根据运行结果和错误信息，修正RPA代码和测试代码。这可以通过编写一个修正函数实现，该函数接收错误信息为输入，输出修正后的RPA代码和测试代码。
   - 步骤6：重复步骤4和步骤5，直到所有的测试代码都能成功执行。这可以通过一个循环实现。

5，算法和数据示例：
   - 算法：使用基于transformer的自然语言处理模型生成RPA任务流程图和伪代码，转化为具体的RPA代码和测试代码。
   - 数据示例：用户输入需求描述“我需要从一个网页上抓取数据并填充到Excel表格中”，输出RPA任务流程图，RPA代码和测试代码。
